In [3]:
import numpy  as np 
import pandas as pd
from processing import *
from scipy.stats import truncnorm
from fastai.tabular.all import *
import matplotlib.pyplot as plt
import seaborn as sn

In [62]:
def get_truncated_normal(mean=265, sd=265, low=0, upp=500):
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [63]:
fs = 256
interval = 20
k = 10

filenames = ['avatar1', 'avengers1', 'bbc1', 'bear1', 'bighero1', 'creed1', 'edgeoftmr11', 'gunviolence1', 'ironman1', 'joe1', 'lex1', 'vox1']
files = [pd.read_csv(('datasets/{}.csvprocessed.csv').format(name)) for name in filenames]

data_sets = pd.concat(files, ignore_index=True).dropna()

In [64]:
labels = data_sets["Interest"]
    
labels_norm = [1 if x == 4 or x == 5 else 0 for x in labels]

data_sets["Interest"] = labels_norm

interested = np.array(data_sets[data_sets['Interest'] == 1])
uninterested = np.array(data_sets[data_sets['Interest'] == 0])

# print(int(len(interested)/(fs*interval))*fs*interval)
# print(int(len(interested)/(fs*interval))) #53
# print(int(len(uninterested)/(fs*interval))) #41


In [65]:
# interestedN = np.array(np.array_split(interested[:int(len(interested)/(fs*interval))*fs*interval], int((len(interested)/(fs*interval))), axis=0))
interestedN1 = np.array(np.array_split(np.array(np.array_split(interested[:int(len(interested)/(fs*interval))*fs*interval], int((len(interested)/(fs*interval))), axis=0)), k, axis=1)).reshape(10*-1, 512, 6)[:, :, 1:5]

sortedN1 = interestedN1[np.argsort(interestedN1.reshape(-1, 512*4).sum(axis=1))]

print(sortedN1.shape)

mean = np.mean(sortedN1.reshape(-1, 512*4))
std = np.std(sortedN1.reshape(-1, 512*4))

mean_indx = find_nearest(np.mean(sortedN1.reshape(-1, 512*4), axis=1), mean)
std_indx = find_nearest(np.mean(sortedN1.reshape(-1, 512*4), axis=1), std)

r = get_truncated_normal(mean=mean_indx, sd=int(std_indx/2), low=0, upp=len(sortedN1))
print(int(r.rvs()))

new_pos_segments = []
for n in range(500):
    ind = [interestedN1[int(r.rvs())] for x in range(10)]
    new_pos_segments.append(np.concatenate(ind, axis=0))

new_pos_segments = np.array(new_pos_segments)
# print(new_pos_segments.shape)

(530, 512, 4)
87


In [66]:
uninterestedN1 = np.array(np.array_split(np.array(np.array_split(uninterested[:int(len(uninterested)/(fs*interval))*fs*interval], int((len(uninterested)/(fs*interval))), axis=0)), k, axis=1)).reshape(10*-1, 512, 6)[:, :, 1:5]

sortedUN1 = uninterestedN1[np.argsort(uninterestedN1.reshape(-1, 512*4).sum(axis=1))]

print(sortedUN1.shape)

Umean = np.mean(sortedUN1.reshape(-1, 512*4))
Ustd = np.std(sortedUN1.reshape(-1, 512*4))

Umean_indx = find_nearest(np.mean(sortedUN1.reshape(-1, 512*4), axis=1), Umean)
Ustd_indx = find_nearest(np.mean(sortedUN1.reshape(-1, 512*4), axis=1), Ustd)

Ur = get_truncated_normal(mean=Umean_indx, sd=int(Ustd_indx/2), low=0, upp=len(sortedUN1))
print(int(Ur.rvs()))


new_neg_segments = []
for n in range(500):
    ind = [uninterestedN1[int(Ur.rvs())] for x in range(10)]
    new_neg_segments.append(np.concatenate(ind, axis=0))

new_neg_segments = np.array(new_neg_segments)
# print(new_neg_segments.shape)

(410, 512, 4)
279


In [67]:
augmented_batches = np.concatenate((new_neg_segments, new_pos_segments), axis=0).reshape(-1*5120, 4, order='F')

print(augmented_batches.shape)

(5120000, 4)


In [68]:
labels = np.concatenate((np.zeros(500), np.full(500, 1)))
# print(labels.shape)

features = PSD(augmented_batches, fs, filtering=True)

# print(features)

normalized = pd.DataFrame(descriptive_stats(features))

  0%|          | 0/4 [00:00<?, ?it/s]

(5120000, 4)


100%|██████████| 1000/1000 [00:15<00:00, 63.92it/s]


In [4]:
normalized["label"] = labels

plt.scatter(normalized.iloc[:, 50], normalized.iloc[:, 40], c=normalized.iloc[:, -1], alpha=0.3, cmap='viridis', s=10)
cor = normalized.iloc[:, 80:].corr()
sn.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

print(normalized.shape)

NameError: name 'labels' is not defined

In [70]:
#Correlation with output variable
cor_target = abs(cor["label"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.01]
relevant_features

82       0.037852
85       0.011438
86       0.043025
93       0.015320
94       0.016204
label    1.000000
Name: label, dtype: float64

In [71]:
procs = [Categorify, FillMissing, Normalize]
dls = TabularDataLoaders.from_df(df = normalized, procs=procs, cont_names=list(normalized.columns)[:-1], 
                                 y_names="label", y_block=CategoryBlock, bs=64)

In [72]:
f1_score = F1Score()
learn = tabular_learner(dls, metrics=[accuracy])


In [73]:
learn.fit_one_cycle(30) # cbs=EarlyStoppingCallback(min_delta=0.1, patience=2)
# learn.recorder.plot_losses()
# algo = svm_model(normalized, labels)
# search = random_search_svm(normalized, labels)
# print(search.best_params_)


epoch,train_loss,valid_loss,accuracy,time
0,0.751621,0.693438,0.550000,00:00
1,0.739701,0.705744,0.535000,00:00
2,0.724767,0.706394,0.525000,00:00
3,0.706171,0.697172,0.545000,00:00
4,0.687338,0.721279,0.500000,00:00
5,0.668575,0.709798,0.560000,00:00
6,0.648058,0.730308,0.495000,00:00
7,0.628691,0.776302,0.500000,00:00
8,0.603965,0.774511,0.505000,00:00
9,0.588936,0.793418,0.500000,00:00


AttributeError: 'TabularModel' object has no attribute 'plot_losses'